In [42]:
# requirements
from pymongo import MongoClient

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA

import joblib

import tqdm
import heapq
import json

In [43]:
# MongoDB 연결
client = MongoClient("mongodb://localhost:3999/")

In [44]:
# DB 불러오기
matchup_statistics_db = client["matchup_statistics_db"]  # summoners, indicators, matches

In [45]:
for collection_name in matchup_statistics_db.list_collection_names():
    print(collection_name)
    collection = matchup_statistics_db["indicators"]

    documents = collection.find()
    cnt = 0

    for document in tqdm.tqdm(documents):
        print(document)
        break
        cnt += 1

    print(cnt)

summoners


0it [00:00, ?it/s]


{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

0it [00:00, ?it/s]


{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

0it [00:00, ?it/s]

{'_id': 't3C2JzNt9v8ypQ3PtL7v4c6VNvn-QdDDnm8-Pcnllyd65xPT', 'matchIndicators': [{'matchId': 'KR_6992198203', 'metadata': {'laneInfo': {'teamPosition': 'BOTTOM', 'isBottomLane': True, 'myTeamId': 100, 'myLaneNumber': 4, 'oppositeLaneNumber': 9, 'myBottomDuoNumber': 5, 'oppositeBottomDuoNumber': 10}, 'timeInfo': {'gameDuration': 1663, 'startTime': 1710722466095, 'endTime': 1710724129616}, 'isFinishedBeforeFifteen': False, 'isOurTeamEarlySurrendered': False, 'isWin': False, 'pingCount': 0, 'champion': 'Smolder'}, 'laneIndicator': {'basicWeight': {'csDiffer': 13, 'expDiffer': 760, 'turretPlateDestroyDiffer': 3.0, 'supportItemFinishedTimeDiffer': 0}, 'aggresiveLaneAbilility': {'dealDiffer': -4025, 'soloKillDiffer': 0, 'duoKillDiffer': -2}, '_class': 'com.ssafy.matchup_statistics.indicator.entity.match.beginning.BtmIndicator'}, 'macroIndicator': {'splitPoint': {'turretKillsPerDeaths': 0, 'damageDealtToTurretsPerTotalDamageDealt': 430, 'damageDealtToTurretsPerTeamTotalTowerDamageDone': 24838}

0


In [55]:
division = {"I": 1, "II": 2, "III": 3, "IV": 4}

emerald_users = []

for summoner in matchup_statistics_db["summoners"].find():
    if summoner["league"]["tier"] == "EMERALD" or summoner["league"]["tier"] == "PLATINUM" or (summoner["league"]["tier"] == "DIAMOND"):
        emerald_users.append(summoner["_id"])

print(len(emerald_users))
print(emerald_users[0])

12744
BExuHZxzPPWkbltQrzVREmaRrRLJ54NMBjlIraqXxdI49HPh


In [56]:
# 학습 데이터 추출

# collection 불러오기
indicators = matchup_statistics_db["indicators"].find()

# 학습 데이터
x = []
y = []

summoners = []

times = 0
all_zero = 0


# 결측치 처리 - 추후에는 평균 값으로 입력
def imputation(datas, indicator, record, record_detail):
    global times

    if datas[indicator][record].get(record_detail):
        return datas[indicator][record].get(record_detail)
    else:
        times += 1
        return 0

# 데이터 추출
for indicator in indicators:
    if indicator["_id"] not in platinum_users:
        continue

    for match_indicator in indicator["matchIndicators"]:
        summoner = {}

        summoner["csDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["csDiffer"]
        summoner["expDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["expDiffer"]
        summoner["turretPlateDestroyDiffer"] = match_indicator["laneIndicator"]["basicWeight"]["turretPlateDestroyDiffer"]
        summoner["dealDiffer"] = imputation(match_indicator, "laneIndicator", "aggresiveLaneAbilility", "dealDiffer")

        summoner["turretKillsPerDeaths"] = match_indicator["macroIndicator"]["splitPoint"]["turretKillsPerDeaths"]
        summoner["damageDealtToTurretsPerTeamTotalTowerDamageDone"] = match_indicator["macroIndicator"]["splitPoint"]["damageDealtToTurretsPerTeamTotalTowerDamageDone"]

        summoner["totalTimeCCingOthersPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalTimeCCingOthersPerTotalDamageTaken"]
        summoner["totalDamageTakenPerTeamTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["totalDamageTakenPerTeamTotalDamageTaken"]
        summoner["damageSelfMitigatedPerTotalDamageTaken"] = match_indicator["macroIndicator"]["initiatingPoint"]["damageSelfMitigatedPerTotalDamageTaken"]

        summoner["visionScorePerDeath"] = match_indicator["macroIndicator"]["visionPoint"]["visionScorePerDeath"]
        summoner["totalJungleObjectivePerGameDuration"] = match_indicator["macroIndicator"]["jungleHoldPoint"]["totalJungleObjectivePerGameDuration"]

        summoner["getObjectiveDifferPerGameDuration"] = match_indicator["macroIndicator"]["objectivePoint"]["getObjectiveDifferPerGameDuration"]

        summoner["damagePerMinute"] = match_indicator["macroIndicator"]["totalDealPoint"]["damagePerMinute"]
        summoner["dealPerGold"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "dealPerGold")
        summoner["teamDamagePercentage"] = imputation(match_indicator, "macroIndicator", "totalDealPoint", "teamDamagePercentage")

        if match_indicator["metadata"]["laneInfo"].get("teamPosition") == "TOP":
            pass


        is_all_zero = True
        idx = 0

        for key, value in summoner.items():
            if value != 0:
                is_all_zero = False
                break 
                
        if is_all_zero is False:

            if match_indicator["metadata"]["isWin"] is True:
                y.append(1)
            else:
                y.append(0)

            summoners.append(summoner)

In [57]:
df = pd.DataFrame(summoners)
df

,csDiffer,expDiffer,turretPlateDestroyDiffer,dealDiffer,turretKillsPerDeaths,damageDealtToTurretsPerTeamTotalTowerDamageDone,totalTimeCCingOthersPerTotalDamageTaken,totalDamageTakenPerTeamTotalDamageTaken,damageSelfMitigatedPerTotalDamageTaken,visionScorePerDeath,totalJungleObjectivePerGameDuration,getObjectiveDifferPerGameDuration,damagePerMinute,dealPerGold,teamDamagePercentage
0,-3,333,0.666667,-164,0,0,50,28430,94592,333333,186,-139,57800000,152525,0.193720
1,5,185,-2.666667,1187,0,0,61,31570,92192,550000,431,323,21500000,56522,0.109684
2,31,871,-4.000000,5435,14285,53157,13,15245,55789,242857,0,-298,128700000,215130,0.377188
3,52,1929,-5.000000,1378,42857,44654,76,17772,49054,157142,0,294,89600000,170786,0.256042
4,27,814,3.000000,1685,28571,72672,60,27469,75273,214285,278,-348,66900000,149614,0.346553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243571,-22,-975,0.000000,374,0,0,143,16661,71062,666666,0,-396,27400000,93863,0.077522
243572,3,639,0.000000,-2168,0,0,292,16979,104833,977777,0,-154,30700000,111993,0.103876
243573,-8,-338,0.000000,4562,0,5541,28,18693,99114,381818,0,0,88700000,219017,0.226356
243574,4,235,0.000000,4169,0,2742,193,17928,55385,516666,0,-65,55200000,224630,0.169108


In [58]:
# PCA
df_normalized = (df - df.min()) / (df.max() - df.min())

In [59]:
scaler = StandardScaler()
x = df_normalized.to_numpy()
x_scaled = scaler.fit_transform(x)

In [51]:
print(x[0])

[4.03448276e-01 3.84128847e-01 6.66666667e-01 5.00091767e-01
 0.00000000e+00 0.00000000e+00 2.38235995e-04 5.27419891e-01
 4.80598711e-02 2.11382114e-02 0.00000000e+00 3.21500000e-01
 3.87035872e-02 7.54666457e-02 1.15233087e-01]


In [60]:
parameter_space = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [16]:
# mlp = MLPClassifier(hidden_layer_sizes=(75, ), max_iter=500, activation='relu', solver='adam', random_state=42, alpha=0.00001, verbose=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(max_iter=500)
random_search = RandomizedSearchCV(mlp, param_distributions=parameter_space, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)
#mlp.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=500), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (50, 50),
                                                               (100, 50)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']},
                   random_state=42, verbose=2)

In [17]:
print("최적의 하이퍼파라미터:", random_search.best_params_)
print("최고 점수:", random_search.best_score_)

최적의 하이퍼파라미터: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'alpha': 0.0001, 'activation': 'tanh'}
최고 점수: 0.8688754608747429


In [61]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=500, activation='tanh', solver='adam', random_state=42, alpha = 0.0001, learning_rate = 'constant', verbose=True)
mlp.fit(x_train, y_train)

Iteration 1, loss = 0.41653892
Iteration 2, loss = 0.33867128
Iteration 3, loss = 0.33340105
Iteration 4, loss = 0.33225934
Iteration 5, loss = 0.33174862
Iteration 6, loss = 0.33095110
Iteration 7, loss = 0.32925196
Iteration 8, loss = 0.32700259
Iteration 9, loss = 0.32402218
Iteration 10, loss = 0.32128187
Iteration 11, loss = 0.31919610
Iteration 12, loss = 0.31796809
Iteration 13, loss = 0.31745084
Iteration 14, loss = 0.31684819
Iteration 15, loss = 0.31643326
Iteration 16, loss = 0.31579802
Iteration 17, loss = 0.31579756
Iteration 18, loss = 0.31506141
Iteration 19, loss = 0.31454097
Iteration 20, loss = 0.31418804
Iteration 21, loss = 0.31378045
Iteration 22, loss = 0.31350072
Iteration 23, loss = 0.31291782
Iteration 24, loss = 0.31252715
Iteration 25, loss = 0.31198835
Iteration 26, loss = 0.31165346
Iteration 27, loss = 0.31102487
Iteration 28, loss = 0.31055515
Iteration 29, loss = 0.31025661
Iteration 30, loss = 0.30977876
Iteration 31, loss = 0.30891646
Iteration 32, los

MLPClassifier(activation='tanh', max_iter=500, random_state=42, verbose=True)

In [62]:
joblib.dump(mlp, 'models/winning/platinum_model.pkl')

['models/winning/platinum_model.pkl']

In [99]:
max_heapq = []

probabilities = mlp.predict_proba(x_test)
cnt = 0
total = 0
for i, probability in enumerate(probabilities):
    heapq.heappush(max_heapq, (-probability[1], i))
    total += 1
    if probability[1] > 0.9:
        cnt += 1
    # print(f"{i}번째 승리확률 : {probability[1]}")

print(total)
print(cnt)

for _ in range(5):
    print(heapq.heappop(max_heapq))

[[0.5480427  0.62149135 0.5        ... 0.13750787 0.296798   0.25519632]
 [0.59786477 0.60837823 0.3        ... 0.27092511 0.38785193 0.63389444]
 [0.51601423 0.52211511 0.5        ... 0.14317181 0.24541629 0.22681683]
 ...
 [0.61209964 0.64615821 0.1        ... 0.34833228 0.40505777 0.61567914]
 [0.59074733 0.57293734 0.5        ... 0.13624921 0.25820276 0.2334923 ]
 [0.49466192 0.49907854 0.6        ... 0.23348018 0.29540348 0.39018069]]
48551
12356
(-0.9999999839766744, 19896)
(-0.9999999779354829, 7970)
(-0.9999999652883452, 8818)
(-0.9999999592109347, 46684)
(-0.9999999435980574, 44015)
